# Baseline: Transfer Learning with ResNet

This notebook implements a baseline approach using transfer learning with a pretrained ResNet model.

Based on research findings:
- Winning solutions use pretrained CNNs (VGG, ResNet, EfficientNet)
- Aggressive data augmentation is key
- Fine-tuning on the Kaggle dataset achieves 98%+ accuracy
- Two-step approach: lightweight model first, then transfer to deeper architecture

## Strategy
1. Use ResNet50 pretrained on ImageNet
2. Resize images to 224x224
3. Apply data augmentation (flips, rotations, color jitter)
4. Fine-tune the top layers
5. Use Adam optimizer with learning rate scheduling

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Check GPU availability
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Load and prepare data
train_dir = '/home/data/train'
test_dir = '/home/data/test'

# Get all training images
train_files = os.listdir(train_dir)
train_files = [f for f in train_files if f.endswith('.jpg')]

# Create labels (cat=0, dog=1)
train_labels = []
for file in train_files:
    if file.startswith('cat'):
        train_labels.append(0)
    elif file.startswith('dog'):
        train_labels.append(1)
    else:
        print(f"Warning: Unknown file {file}")

print(f"Total training images: {len(train_files)}")
print(f"Cats: {sum(1 for x in train_labels if x == 0)}")
print(f"Dogs: {sum(1 for x in train_labels if x == 1)}")

# Split into train/validation
train_files, val_files, train_labels, val_labels = train_test_split(
    train_files, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

print(f"Train set: {len(train_files)} images")
print(f"Validation set: {len(val_files)} images")

In [ ]:
# Define custom dataset
class DogsCatsDataset(Dataset):
    def __init__(self, file_names, labels, transform=None, is_test=False):
        self.file_names = file_names
        self.labels = labels
        self.transform = transform
        self.is_test = is_test
        self.data_dir = test_dir if is_test else train_dir
    
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.file_names[idx])
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:
            return image
        else:
            return image, self.labels[idx]

# Define transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = DogsCatsDataset(train_files, train_labels, transform=train_transform)
val_dataset = DogsCatsDataset(val_files, val_labels, transform=val_transform)

# Create dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Train batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

In [ ]:
# Create model
model = models.resnet50(pretrained=True)

# Freeze all layers except the final layer
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_features, 2)
)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"Model loaded on: {device}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

In [ ]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5, verbose=True)

# Training function
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training')
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({
            'Loss': f'{running_loss/total*batch_size:.4f}',
            'Acc': f'{100.*correct/total:.2f}%'
        })
    
    return running_loss / len(loader), 100. * correct / total

# Validation function
def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation')
        for images, labels in pbar:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            pbar.set_postfix({
                'Loss': f'{running_loss/total*batch_size:.4f}',
                'Acc': f'{100.*correct/total:.2f}%'
            })
    
    return running_loss / len(loader), 100. * correct / total

In [ ]:
# Train the model
num_epochs = 10
best_val_acc = 0.0

history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 50)
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
    
    scheduler.step(val_loss)
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), '/home/code/experiments/001_baseline_transfer_learning/best_model.pth')
        print(f"New best model saved with val accuracy: {val_acc:.2f}%")

print(f"\nTraining completed! Best validation accuracy: {best_val_acc:.2f}%")

In [ ]:
# Load best model and make predictions on test set
model.load_state_dict(torch.load('/home/code/experiments/001_baseline_transfer_learning/best_model.pth'))
model.eval()

# Get test files
test_files = sorted(os.listdir(test_dir))
test_files = [f for f in test_files if f.endswith('.jpg')]

# Create test dataset and loader
test_dataset = DogsCatsDataset(test_files, [0]*len(test_files), transform=val_transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Make predictions
predictions = []
with torch.no_grad():
    for images in tqdm(test_loader, desc='Predicting'):
        images = images.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)[:, 1]  # Probability of being a dog
        predictions.extend(probs.cpu().numpy())

print(f"Generated {len(predictions)} predictions")
print(f"Sample predictions: {predictions[:5]}")

In [ ]:
# Create submission file
submission = pd.DataFrame({
    'id': range(1, len(predictions) + 1),
    'label': predictions
})

submission_path = '/home/submission/submission.csv'
submission.to_csv(submission_path, index=False)

print(f"Submission saved to: {submission_path}")
print(f"Submission shape: {submission.shape}")
print(f"\nFirst 10 rows:")
print(submission.head(10))
print(f"\nLast 10 rows:")
print(submission.tail(10))